**Criando conexão no sistema.**

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col


# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()


/usr/local/lib/python3.12/dist-packages/google/cloud/dataproc_spark_connect/session.py:332: UserWarning: Python version mismatch detected: Client is using Python 3.12, but Dataproc runtime 3.0 uses Python 3.11. This mismatch may cause issues with Python UDF (User Defined Function) compatibility. Consider using Python 3.11 for optimal UDF execution.
  dataproc_config: Session = self._get_dataproc_config()


███████████████████████████████████████████▋                                    

**Lendo tabela da bronze**

In [ ]:
df = spark.read.format("bigquery").option("table", "clinic-de.1_bronze.samples").load()

df.printSchema()

root
 |-- timeline: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- event: string (nullable = true)
 |    |    |-- ts: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- specimen_type: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- test_code: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- sample_id: string (nullable = true)



**Utilizando a função exploded na coluna "timeline"**

In [ ]:
df_exploded = df.withColumn("timeline", explode("timeline"))

df_tratado_v1 = df_exploded.select(
    "sample_id",
    "timeline.event",
    "timeline.ts"
)



**Salvando dataframe na silver.**

In [ ]:
(df_tratado_v1.write
        .format("bigquery")
        .option("table", "clinic-de.2_silver.samples_timeline")
        .option("writeMethod", "direct")
        .mode("overwrite")
        .save()
    )

print("Tabela salva com sucesso")

  0%|           0/1 Tasks

Tabela salva com sucesso


In [ ]:
spark.stop()